In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
import os, glob
from nltk.corpus import stopwords

In [9]:
def clean(text):
    return unicode(''.join([i if ord(i) < 128 else ' ' for i in text]))

In [10]:
import spacy
from spacy.lang.en import English
nlp = spacy.load('en_core_web_sm')

In [11]:
def preprocessing(text):
    doc = clean(text)

    doc = str(doc).lower()
    doc = nlp(clean(doc))
    
    # we add some words to the stop word list
    texts, article = '', []
    for w in doc:
        if str(w) != '\n':
            texts = texts + ' ' + str(w.text)


    i = 0
    while i < len(texts):
        if((ord(texts[i])>=65 and ord(texts[i]) <=90) or ord(texts[i]) == 46):
            pass
        elif((ord(texts[i])>=97 and ord(texts[i]) <=122) or ord(texts[i]) ==95 or ord(texts[i]) == 32):
            pass
        else:
            texts = texts.replace(texts[i], '')
        i +=1
            #texts.remove(texts[i])
            
    return texts


def noun_adj(texts):

    tokens = nltk.sent_tokenize(texts)
    #print len(tokens)
    keyphrases = []
    for i in range(len(tokens)):
        tok = nltk.word_tokenize(tokens[i])
        #print tok
        pos = nltk.pos_tag(tok)
        #print pos
        #print pos
        l, ll = '', []
        for word, pos in pos:
            if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'JJ' or pos == 'JJR' or pos == 'JJS'):
                if len(l) != 0:
                    l = l + " " + word
                else:
                    l = word
            elif len(l) != 0:
                ll.append(l)
                l = ''
        if len(l) != 0:
            ll.append(l)
            l = ''
        keyphrases.append(ll)
        ll = []

    return keyphrases

def mainKeyphrases(keyphrases):
        #not w.is_stop and not w.is_punct and not w.like_num
    #print ll
    #print nltk.word_tokenize(ll[9])
    mainKeys, ke, kk = [], [], []
    #for i in range(len(keyphrases)):
    i=0
    while i < len(keyphrases):
        #############
        if len(keyphrases[i]) == 0:
            del keyphrases[i]
            i -= 1
        else:
            for j in range(len(keyphrases[i])):
                    #print i,j
                    #print keyphrases[i][j]
                kk =  nltk.word_tokenize(keyphrases[i][j])
                    #print kk
                k=0
                while k < len(kk):
                    ####################3
                    if len(kk[k]) < 3:
                        del kk[k]
                        k -= 1
                    k+=1
                            
                ke.append(kk)
            mainKeys.extend(ke)
            ke = []
        i += 1
    #print mainKeys
    
    #joining keyphrases
    keyphrase = []
    for i in mainKeys:
        keyphrase.append(' '.join(i))
        
    #removing empty items
    i=0
    while i < len(keyphrase):
        if len(keyphrase[i]) < 1:
            keyphrase.remove(keyphrase[i])
            i-=1
        i+=1
    
    return keyphrase

In [12]:
def keyphrase_extraction(text, n):
    
    vec = TfidfVectorizer()
    tfidf_vec = vec.fit_transform((str(x) for x in text))
    
    ranking = list()
    for i, c in enumerate(tfidf_vec):
        ranking.append((i, c.sum()))
        
    top_sentences = sorted(ranking, key=lambda x: x[1], reverse=True)
    
    result = []
    for i in top_sentences:
        result.append(text[i[0]])
        
    seen = set()
    result = [x for x in result if not (x in seen or seen.add(x))]
    return result[:n]

In [19]:
lee_train_file = "art_and_culture.txt"
with open(lee_train_file) as f:
    story = f.read()
    texts = preprocessing(story)
    filtered = noun_adj(texts)
    phrases = mainKeyphrases(filtered)
print keyphrase_extraction(phrases, 5)

['canadian pop star michael buble married argentine actress luisana lopilato', 'buble weds star luisana lopilato canadian pop star michael buble', 'north america top grossing concert entertainers', 'bride argentine actress luisana lopilato', 'rebel way little girls']
